In [ ]:
r = requests.get(url)

soup = bs(r.text, "lxml")

table = soup.find(id = "GridView1")
rows = table.find_all("tr")

var_names = []
var_descrs = []
file_names = []
file_descrs = []
start_years = []
end_years = []
components = []
constraints = []


for row in range(1, len(rows)):
    var_name, var_descr, file_name, file_descr, start_year, end_year, component, constraint = rows[row].find_all("td")

    var_names.append(var_name.text)
    var_descrs.append(var_descr.text)
    file_names.append(file_name.text)
    file_descrs.append(file_descr.text)
    start_years.append(start_year.text)
    end_years.append(end_year.text)
    components.append(component.text)
    constraints.append(constraint.text)


df = pd.DataFrame(list(zip(var_names, var_descrs, file_names, file_descrs, start_years, end_years, components, constraints)), columns = ["var_name", "var_descr", "file_name", "file_descr", "start_year", "end_year", "component", "constraint"])

df.head(2)

In [ ]:
def var_descr_detector(var_name, vars_df):
    descr = vars_df[vars_df["vAr_nAmE"] == var_name]["var_descr"].values[0]
    return descr

def n_rows(df, n_columns):
    columns = list(df.columns)

    if len(columns) % n_columns == 0:
        axes_rows = len(columns) // n_columns
    else:
        axes_rows = (len(columns) // n_columns) + 1

    return axes_rows

def multi_axes_plotter(df, n_columns, kind, figsize, var_names = None):
    n_rows_ = n_rows(df, n_columns)

    fig, axes = plt.subplots(n_rows_, n_columns, figsize = figsize)
    count = 0

    for row in range(axes.shape[0]):
        for column in range(axes.shape[1]):
            if kind == "strip":
                sns.stripplot(y = df.iloc[:, count], ax = axes[row][column])
            elif kind == "dist":
                sns.distplot(df.iloc[:, count], ax = axes[row][column])
            elif kind == "box":
                sns.boxplot(df.iloc[:, count], ax = axes[row][column])
            else:
                sns.histplot(df.iloc[:, count], ax = axes[row][column], bins = 30)

            try:
                axes[row][column].set(xlabel = var_descr_detector(df.iloc[:, count].name, var_names))
            except:
                pass

            if (count + 1) < df.shape[1]:
                count += 1
            else:
                break

    return fig